# RAG Query System Testing

This notebook tests the RAG (Retrieval Augmented Generation) system for querying climate literature.

## System Overview

- **Retrieval**: SQLAlchemy + pgvector for semantic search
- **Synthesis**: GPT-4o for generating responses
- **Data**: Document chunks stored in PostgreSQL from lit_mining_v3.ipynb

## Prerequisites

1. Document chunks must be uploaded to PostgreSQL (run lit_mining_v3.ipynb first)
2. DATABASE_URL must be set in `.env` file
3. OpenAI API key must be set in environment

## Setup and Initialization

In [9]:
# Import required libraries
from rag_query_system import ClimateRAGSystem
import os
from dotenv import load_dotenv
import json

# Load environment variables
load_dotenv('../.env')

# Verify environment
DATABASE_URL = os.getenv("DATABASE_URL")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not DATABASE_URL:
    raise ValueError("DATABASE_URL not found in environment")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY not found in environment")

print("✅ Environment variables loaded")
print(f"   Database: {DATABASE_URL.split('@')[1] if '@' in DATABASE_URL else 'configured'}")

✅ Environment variables loaded
   Database: localhost:5432/climate_viewer_dev


In [10]:
# Initialize the RAG system
rag_system = ClimateRAGSystem(
    database_url=DATABASE_URL,
    model="gpt-4o",
    embedding_model="text-embedding-3-small"
)

print("✅ RAG system initialized")
print(f"   Model: gpt-4o")
print(f"   Embedding model: text-embedding-3-small")

✅ RAG system initialized
   Model: gpt-4o
   Embedding model: text-embedding-3-small


## System Health Check

In [11]:
from sqlalchemy import create_engine, text

engine = create_engine(DATABASE_URL)

with engine.connect() as conn:
    result = conn.execute(text("SELECT COUNT(*) FROM document_chunks"))
    total_chunks = result.scalar()
    
    result = conn.execute(text("SELECT COUNT(*) FROM document_chunks WHERE embedding IS NOT NULL"))
    chunks_with_embeddings = result.scalar()
    
    result = conn.execute(text("""
        SELECT DISTINCT unnest(relevant_layers) as layer 
        FROM document_chunks 
        WHERE relevant_layers IS NOT NULL
        ORDER BY layer
    """))
    layers = [row[0] for row in result]

print("📊 Database Status:")
print(f"   Total chunks: {total_chunks}")
print(f"   Chunks with embeddings: {chunks_with_embeddings}")
print(f"\n📁 Available Layers ({len(layers)}):")
for layer in layers:
    print(f"   - {layer}")

📊 Database Status:
   Total chunks: 568
   Chunks with embeddings: 568

📁 Available Layers (8):
   - annual_high_wave_flooding
   - compound_flooding
   - drainage_backflow
   - emergent_and_shallow_groundwater
   - future_erosion_hazard_zone
   - groundwater_inundation
   - low_lying_flooding
   - passive_marine_flooding


## Test 1: Groundwater Inundation

In [12]:
query = "What are the projected impacts of groundwater inundation in Hawaii due to sea level rise?"

result = rag_system.generate_response(
    query=query,
    top_k=10,
    layers=["groundwater_inundation"],
    min_confidence="MEDIUM",
    temperature=0.3
)

rag_system.print_response(result)

🔍 Retrieving top 10 chunks...
   Filtering by layers: groundwater_inundation
✅ Retrieved 10 chunks
📝 Building context prompt...
🤖 Generating response with gpt-4o...

ANSWER
Groundwater inundation in Hawaii due to sea level rise is a significant concern, especially for urban areas like Honolulu and Waikiki. As sea levels rise, groundwater, which is the water found beneath the Earth's surface, can also rise and lead to flooding. This type of flooding can be particularly tricky because it doesn't just come from the ocean; it seeps up from below the ground.

In Honolulu, for example, research indicates that with a sea level rise of about 0.98 meters (or roughly 3.2 feet) by the end of the century, around 42% of areas like Waikiki could experience chronic groundwater flooding. This means that the water table will rise to the surface, causing flooding even without rain or storms. This could affect roads, buildings, and even sewage systems, leading to potential contamination issues.

The impa

## Test 2: Coastal Erosion

In [ ]:
query = "What are the observed and projected erosion rates for Hawaiian beaches?"

result = rag_system.generate_response(
    query=query,
    top_k=8,
    layers=["future_erosion_hazard_zone"],
    min_confidence="HIGH",
    temperature=0.2
)

rag_system.print_response(result)

## Test 3: Multi-Layer Flooding

In [ ]:
query = "What types of flooding will affect Honolulu and Waikiki with 3 feet of sea level rise?"

result = rag_system.generate_response(
    query=query,
    top_k=15,
    layers=["passive_marine_flooding", "groundwater_inundation", "drainage_backflow", "compound_flooding"],
    min_confidence="MEDIUM",
    temperature=0.3
)

rag_system.print_response(result)

## Test 4: Open Query (No Filters)

In [ ]:
query = "What infrastructure in Hawaii is most vulnerable to sea level rise by 2050?"

result = rag_system.generate_response(
    query=query,
    top_k=12,
    layers=None,
    min_confidence="MEDIUM",
    temperature=0.3
)

rag_system.print_response(result)

## Test 5: Auto-Detection of Layers from Keywords

The system can automatically detect relevant layers based on keywords in your query!

In [ ]:
# Example 1: Query with "groundwater" keyword - should auto-detect groundwater_inundation layer
query = "What is passive marine flooding?"

result = rag_system.generate_response(
    query=query,
    top_k=10,
    layers=None,  # No manual layer specification (auto_detect_layers=True is the default)
    min_confidence="MEDIUM",
    temperature=0.3
)

rag_system.print_response(result)

In [ ]:
# Example 2: Query with "erosion" keyword - should auto-detect future_erosion_hazard_zone
query = "What are the beach erosion rates on Oahu and Maui?"

result = rag_system.generate_response(
    query=query,
    top_k=8,
    # Note: layers=None and auto_detect_layers=True are defaults
    temperature=0.3
)

rag_system.print_response(result)

In [ ]:
# Example 3: Query with "wave" keyword - should auto-detect annual_high_wave_flooding
query = "What is the impact of wave runup and high waves on coastal infrastructure?"

result = rag_system.generate_response(
    query=query,
    top_k=10,
    temperature=0.3
)

rag_system.print_response(result)

In [ ]:
# Example 4: Multiple keywords - should detect multiple layers
query = "What are the combined effects of storm drain backflow and coastal flooding in Honolulu?"

result = rag_system.generate_response(
    query=query,
    top_k=15,  # Retrieve more chunks since we're covering multiple layers
    temperature=0.3
)

rag_system.print_response(result)

### Testing the Layer Detection Function Directly

You can also test the layer detection without running a full query:

In [ ]:
# Test layer detection on various queries
test_queries = [
    "What is the impact of groundwater on buildings?",
    "How fast are beaches eroding?",
    "What happens during high wave events?",
    "Will storm drains overflow with sea level rise?",
    "What areas are below critical elevation thresholds?",
    "How does compound flooding work?",
]

print("Testing Layer Auto-Detection:\n")
print("=" * 80)

for query in test_queries:
    detected = rag_system.detect_layers_from_query(query)
    print(f"\nQuery: {query}")
    if detected:
        print(f"Detected Layers: {', '.join(detected)}")
    else:
        print("Detected Layers: None (will search all layers)")
    print("-" * 80)

## Advanced: Direct Chunk Retrieval

In [ ]:
query = "groundwater flooding in urban areas"

chunks = rag_system.retrieve_chunks(
    query=query,
    top_k=5,
    layers=["groundwater_inundation"],
    min_confidence="HIGH"
)

print(f"Retrieved {len(chunks)} chunks:\n")

for i, chunk in enumerate(chunks, 1):
    print(f"[{i}] {chunk['filename']}")
    print(f"    Similarity: {chunk['similarity_score']:.4f}")
    print(f"    Confidence: {chunk['confidence']}")
    print(f"    Text: {chunk['text'][:200]}...\n")

## Custom Query Cell

In [ ]:
# Write your own custom query
query = "What is the definition of passive marine flooding?"

result = rag_system.generate_response(
    query=query,
    top_k=10,
    layers=None,
    min_confidence="MEDIUM",
    temperature=0.3,
    auto_detect_layers=True
)

rag_system.print_response(result)

🔍 Retrieving top 10 chunks...
✅ Retrieved 10 chunks
📝 Building context prompt...
🤖 Generating response with gpt-4o...

ANSWER
Passive marine flooding, also known as marine inundation, is when rising sea levels cause ocean water to spill over onto the land. Imagine it like a bathtub slowly filling up and eventually overflowing. This type of flooding doesn't need a storm or big waves to happen; it occurs gradually as sea levels rise over time.

In Hawaii, this is a concern because as sea levels continue to rise, areas that are close to the coast, like Waikiki in Honolulu, could see regular flooding. This could happen even during calm weather, especially during high tides. The University of Hawaii's research models this flooding using a "bathtub" approach to identify low-lying areas that might be affected. They look at scenarios of sea level rise from 0 to 10 feet to understand how different levels might impact the islands.

For example, with just a few feet of sea level rise, some coasta

In [ ]:
# Write your own custom query
query = "What is groundwater flooding?"

result = rag_system.generate_response(
    query=query,
    top_k=10,
    layers=None,
    min_confidence="MEDIUM",
    temperature=0.3,
    auto_detect_layers=False
)

rag_system.print_response(result)

🔍 Retrieving top 10 chunks...
✅ Retrieved 10 chunks
📝 Building context prompt...
🤖 Generating response with gpt-4o...

ANSWER
Groundwater flooding happens when the water table, which is the upper level of groundwater, rises to the point where it reaches or exceeds the ground surface. This can lead to flooding from below, rather than from rain or ocean water coming in from above. In places like Honolulu and Waikiki, this is a growing concern because as sea levels rise, the groundwater levels rise too. This is because groundwater and seawater are connected, especially in coastal areas.

For example, in Waikiki, about 42% of the area has groundwater depths shallower than 1.3 meters (about 4 feet). This means that as sea levels continue to rise, the risk of groundwater flooding increases, potentially affecting roads, basements, and even sewage systems. In fact, nearly 90% of On-Site Sewage Disposal Systems (OSDSs) in Honolulu are already compromised during king-tide events, which are excep

## Test 6: Definition Responses for All Layers

Generating definition responses for each layer type to understand what each represents.


### Annual High Wave Flooding


In [13]:
# Write your own custom query
query = "What is the definition of annual high wave flooding?"

result = rag_system.generate_response(
    query=query,
    top_k=10,
    layers=None,
    min_confidence="MEDIUM",
    temperature=0.0,
    auto_detect_layers=True
)

rag_system.print_response(result)


🔍 Auto-detected layers: annual_high_wave_flooding
🔍 Retrieving top 10 chunks...
   Filtering by layers: annual_high_wave_flooding
✅ Retrieved 10 chunks
📝 Building context prompt...
🤖 Generating response with gpt-4o...

ANSWER
Annual high wave flooding is when large waves, often driven by storms or seasonal swells, cause coastal areas to flood. Imagine the ocean getting a bit more energetic than usual, with waves that are bigger and more powerful. These waves can push water onto the land, especially in places like Hawaii where the coastline is close to sea level. 

This type of flooding is particularly important because it can happen even if the weather seems calm inland. For example, during the winter months, Hawaii often experiences large swells from the North Pacific, which can lead to high wave flooding on north-facing shores of islands like O'ahu and Kaua'i. 

The impact of this flooding can be significant, affecting roads, homes, and businesses near the coast. It's not just about 

### Compound Flooding


In [14]:
# Write your own custom query
query = "What is the definition of compound flooding?"

result = rag_system.generate_response(
    query=query,
    top_k=10,
    layers=None,
    min_confidence="MEDIUM",
    temperature=0.0,
    auto_detect_layers=True
)

rag_system.print_response(result)


🔍 Auto-detected layers: compound_flooding
🔍 Retrieving top 10 chunks...
   Filtering by layers: compound_flooding
✅ Retrieved 2 chunks
📝 Building context prompt...
🤖 Generating response with gpt-4o...

ANSWER
Compound flooding is when different types of flooding happen at the same time or in quick succession, making the overall impact much worse. Imagine a big rainstorm hitting Hawaii just as the ocean is experiencing a high tide. The rainwater has nowhere to go because the ocean is already so high, so it floods the land even more. This is what happened during the Kona storm from December 5-7, 2021, in areas like Kona Moku and Honolulu. During that storm, heavy rain, a King tide, and an 8-inch storm surge all combined to cause significant flooding.

In Hawaii, compound flooding is a real concern because of the way different factors can come together. As sea levels rise—potentially up to 10 feet in the future—these events could become more frequent and severe. This means that places lik

### Drainage Backflow


In [15]:
# Write your own custom query
query = "What is the definition of drainage backflow?"

result = rag_system.generate_response(
    query=query,
    top_k=10,
    layers=None,
    min_confidence="MEDIUM",
    temperature=0.0,
    auto_detect_layers=True
)

rag_system.print_response(result)


🔍 Auto-detected layers: drainage_backflow
🔍 Retrieving top 10 chunks...
   Filtering by layers: drainage_backflow
✅ Retrieved 10 chunks
📝 Building context prompt...
🤖 Generating response with gpt-4o...

ANSWER
Drainage backflow is when storm drains and sewers can't handle the amount of water during a flood, causing water to flow back out onto the streets instead of draining away. Imagine it like a sink overflowing because the drain is clogged. This can happen during heavy rains or high tides, especially in places like Waikīkī in Honolulu, where the drainage system is already under pressure from rising sea levels. As sea levels rise, the problem of drainage backflow is expected to get worse, leading to more frequent and severe flooding in low-lying areas. This is a concern for residents and businesses because it can lead to water damage and disrupt daily life. Scientists are working on solutions, but it's a complex issue that requires careful planning and adaptation.

SOURCES (10 docume

### Emergent and Shallow Groundwater


In [16]:
# Write your own custom query
query = "What is the definition of emergent and shallow groundwater?"

result = rag_system.generate_response(
    query=query,
    top_k=10,
    layers=None,
    min_confidence="MEDIUM",
    temperature=0.0,
    auto_detect_layers=True
)

rag_system.print_response(result)


🔍 Auto-detected layers: groundwater_inundation, emergent_and_shallow_groundwater
🔍 Retrieving top 10 chunks...
   Filtering by layers: groundwater_inundation, emergent_and_shallow_groundwater
✅ Retrieved 10 chunks
📝 Building context prompt...
🤖 Generating response with gpt-4o...

ANSWER
Emergent and shallow groundwater refers to groundwater that is very close to the surface of the land, sometimes even reaching the surface itself. This can happen in coastal areas like Honolulu and Waikiki in Hawaii, where the water table is naturally high due to the proximity to the ocean. As sea levels rise, this groundwater can rise too, leading to flooding from below the ground rather than from the ocean or rain.

In places like Waikiki, about 42% of the area already has groundwater depths shallower than 1.3 meters (or about 4 feet). This means that during high tides or heavy rain, the water can easily reach the surface, causing flooding. This type of flooding is particularly concerning because it ca

### Future Erosion Hazard Zone


In [39]:
# Write your own custom query
query = "What is the definition of a future erosion hazard zone?"

result = rag_system.generate_response(
    query=query,
    top_k=10,
    layers=None,
    min_confidence="MEDIUM",
    temperature=0.0,
    auto_detect_layers=True
)

rag_system.print_response(result)


🔍 Auto-detected layers: future_erosion_hazard_zone
🔍 Retrieving top 10 chunks...
   Filtering by layers: future_erosion_hazard_zone
✅ Retrieved 10 chunks
📝 Building context prompt...
🤖 Generating response with gpt-4o...

ANSWER
A "future erosion hazard zone" is an area along the coast where the shoreline is expected to move inland due to erosion over time. This happens because of factors like rising sea levels and changes in wave patterns, which gradually wear away the land. In Hawaii, this is a big concern because many of the islands' beautiful beaches and important infrastructure are close to the ocean.

For example, research shows that by 2050, about 92% of Hawaiian shorelines (excluding Kailua) are projected to retreat, and by 2100, this could increase to 96%. This means that the beaches and land we see today might not be there in the future, affecting homes, roads, and natural habitats. In some places, the shoreline could move inland by as much as 60 meters (about 197 feet) by 210

### Low Lying Flooding


In [ ]:
# Write your own custom query
query = "What is the definition of low-lying flooding?"

result = rag_system.generate_response(
    query=query,
    top_k=10,
    layers=None,
    min_confidence="MEDIUM",
    temperature=0.0,
    auto_detect_layers=True
)

rag_system.print_response(result)


🔍 Retrieving top 10 chunks...
✅ Retrieved 10 chunks
📝 Building context prompt...
🤖 Generating response with gpt-4o...

ANSWER
Low-lying flooding refers to the flooding that occurs in areas that are at a low elevation, making them particularly vulnerable to rising water levels. Imagine a flat, low-lying area near the coast; as sea levels rise, these areas are more likely to experience flooding because they are closer to the water level. This type of flooding can happen even if the area isn't directly connected to the ocean, as water can seep through the ground or come from nearby bodies of water.

In Hawaii, low-lying flooding is a concern because many coastal areas are at risk as sea levels continue to rise. For example, with projections of up to 3.2 feet of sea level rise by mid-century, areas like Waikīkī and parts of Honolulu could see significant flooding. This could affect homes, roads, and other infrastructure, making it a critical issue for local communities.

It's important to 

## Test 7: Definition Responses Without Auto-Detection

Generating definition responses for each layer type with explicit layer specification (no auto-detection).


### Passive Marine Flooding (No Auto-Detection)


In [19]:
# Write your own custom query
query = "What is the definition of passive marine flooding?"

result = rag_system.generate_response(
    query=query,
    top_k=10,
    layers=None,
    min_confidence="MEDIUM",
    temperature=0.0,
    auto_detect_layers=False
)

rag_system.print_response(result)


🔍 Retrieving top 10 chunks...
✅ Retrieved 10 chunks
📝 Building context prompt...
🤖 Generating response with gpt-4o...

ANSWER
Passive marine flooding, also known as marine inundation, is when rising sea levels cause ocean water to overflow onto the land. Imagine it like a bathtub slowly filling up with water until it spills over the edge. This type of flooding doesn't need a storm or big waves to happen; it's a gradual process that can become permanent as sea levels continue to rise. 

In Hawaii, this is a concern because as the ocean rises, low-lying coastal areas can start to flood regularly. For example, with sea level rise projections of up to 10 feet, areas that are currently dry could be underwater in the future. This could affect places like Waikiki and other coastal communities, impacting homes, roads, and businesses. 

Scientists use models to predict how much and where this flooding might occur, but there's still some uncertainty. Factors like local topography and future sea 

### Groundwater Inundation (No Auto-Detection)


In [20]:
# Write your own custom query
query = "What is the definition of groundwater flooding?"

result = rag_system.generate_response(
    query=query,
    top_k=10,
    layers=None,
    min_confidence="MEDIUM",
    temperature=0.0,
    auto_detect_layers=False
)

rag_system.print_response(result)


🔍 Retrieving top 10 chunks...
✅ Retrieved 10 chunks
📝 Building context prompt...
🤖 Generating response with gpt-4o...

ANSWER
Groundwater flooding happens when the water table, which is the upper level of groundwater in the soil, rises to the point where it reaches or exceeds the ground surface. This can lead to flooding from below, rather than from rain or ocean water coming in from above. In places like Honolulu and Waikiki, this is a growing concern because as sea levels rise, the groundwater levels also rise. 

For example, research shows that in Waikiki, about 42% of the area already has groundwater depths shallower than 1.3 meters. This means that as sea levels continue to rise, the groundwater will push up even more, potentially causing flooding in areas that might not be directly affected by ocean water. This type of flooding can affect roads, buildings, and even underground infrastructure like sewage systems, leading to contamination and other issues.

It's important to note t

### Annual High Wave Flooding (No Auto-Detection)


In [21]:
# Write your own custom query
query = "What is the definition of annual high wave flooding?"

result = rag_system.generate_response(
    query=query,
    top_k=10,
    layers=None,
    min_confidence="MEDIUM",
    temperature=0.0,
    auto_detect_layers=False
)

rag_system.print_response(result)


🔍 Retrieving top 10 chunks...
✅ Retrieved 10 chunks
📝 Building context prompt...
🤖 Generating response with gpt-4o...

ANSWER
Annual high wave flooding refers to the type of coastal flooding that happens when large waves, often from ocean swells, crash onto the shore and push water further inland than usual. This kind of flooding is particularly common in places like Hawaii, where the ocean can generate significant wave heights, especially during certain times of the year when swells are more frequent.

Here's how it works: Imagine a big wave rolling in from the ocean. When it reaches the shore, it doesn't just stop there. The energy from the wave can cause water to surge up the beach, sometimes overtopping barriers like seawalls. This is known as wave runup. If the waves are big enough and the conditions are right, this can lead to flooding in coastal areas.

In Hawaii, the annually recurring significant wave height for open North and Northwest Pacific swells is about 25 feet. During 

### Compound Flooding (No Auto-Detection)


In [22]:
# Write your own custom query
query = "What is the definition of compound flooding?"

result = rag_system.generate_response(
    query=query,
    top_k=10,
    layers=None,
    min_confidence="MEDIUM",
    temperature=0.0,
    auto_detect_layers=False
)

rag_system.print_response(result)


🔍 Retrieving top 10 chunks...
✅ Retrieved 10 chunks
📝 Building context prompt...
🤖 Generating response with gpt-4o...

ANSWER
Compound flooding is when different types of flooding happen at the same time or in quick succession, making the overall impact much worse. In Hawaii, this often means heavy rainfall coinciding with high tides, which can lead to more severe flooding than if each event happened separately. For example, during a storm, you might have rainwater flooding streets while the ocean is also pushing water onto the land because of high tides. This combination can overwhelm drainage systems and lead to significant flooding in areas like Kona Moku and Honolulu.

To give you an idea of the scale, during a historical event in December 2021, a storm in Kona Moku combined with a King tide and an 8-inch storm surge, leading to widespread flooding. Researchers have used this event to simulate future scenarios, showing how flooding could increase with sea level rise. They predict t

### Drainage Backflow (No Auto-Detection)


In [23]:
# Write your own custom query
query = "What is the definition of drainage backflow?"

result = rag_system.generate_response(
    query=query,
    top_k=10,
    layers=None,
    min_confidence="MEDIUM",
    temperature=0.0,
    auto_detect_layers=False
)

rag_system.print_response(result)


🔍 Retrieving top 10 chunks...
✅ Retrieved 10 chunks
📝 Building context prompt...
🤖 Generating response with gpt-4o...

ANSWER
Drainage backflow is when water from the ocean or other bodies of water flows back into storm drains and sewers, instead of draining away as it should. This usually happens in low-lying coastal areas, like parts of Honolulu, where the elevation difference between the drainage system and the ocean is small. During high tides or heavy rain, the water level can rise enough to reverse the flow, causing flooding in streets and basements. It's like when you pour too much water into a sink and it starts to spill over because it can't drain fast enough. This type of flooding can be particularly problematic because it can happen even without a storm, just from regular tidal changes. In Honolulu, this is a concern because it can lead to flooded streets and infrastructure, especially during king tides or when sea levels rise. Scientists are working on solutions like instal

### Emergent and Shallow Groundwater (No Auto-Detection)


In [24]:
# Write your own custom query
query = "What is the definition of emergent and shallow groundwater?"

result = rag_system.generate_response(
    query=query,
    top_k=10,
    layers=None,
    min_confidence="MEDIUM",
    temperature=0.0,
    auto_detect_layers=False
)

rag_system.print_response(result)


🔍 Retrieving top 10 chunks...
✅ Retrieved 10 chunks
📝 Building context prompt...
🤖 Generating response with gpt-4o...

ANSWER
Emergent and shallow groundwater refers to groundwater that is very close to the surface of the land, sometimes even reaching the surface itself. This can happen in coastal areas like Honolulu and Waikiki in Hawaii, where the groundwater levels are influenced by the ocean's tides and long-term sea level rise. 

In simple terms, think of it like this: the ground is like a sponge, and when the sea level rises, it pushes the water in the sponge closer to the surface. In places like Waikiki, about 42% of the area has groundwater less than 1.3 meters (or about 4 feet) below the surface. This means that even a small rise in sea level can cause the groundwater to emerge, leading to flooding from below, not just from the ocean.

This is important because it can affect buildings, roads, and other infrastructure. For example, nearly 90% of on-site sewage systems in Honolu

### Future Erosion Hazard Zone (No Auto-Detection)


In [40]:
# Write your own custom query
query = "What is the definition of a future erosion hazard zone?"

result = rag_system.generate_response(
    query=query,
    top_k=10,
    layers=None,
    min_confidence="MEDIUM",
    temperature=0.0,
    auto_detect_layers=False
)

rag_system.print_response(result)


🔍 Retrieving top 10 chunks...
✅ Retrieved 10 chunks
📝 Building context prompt...
🤖 Generating response with gpt-4o...

ANSWER
A "future erosion hazard zone" is an area along the coast where the shoreline is expected to erode or move inland over time due to factors like rising sea levels and increased wave activity. Imagine the beach slowly creeping closer to homes and roads as the ocean eats away at the land. This is a big concern for places like Hawaii, where many communities and important infrastructure are near the coast.

In Hawaii, studies show that a significant portion of the shoreline is already eroding. For example, about 70% of sandy shorelines on Oahu, Maui, and Kauai are experiencing chronic erosion. By 2050, it's projected that 92% of Hawaiian shorelines (excluding Kailua) will retreat, and by 2100, this number could rise to 96%. This means that many beaches could move inland by several meters, potentially affecting homes, roads, and other structures.

The exact amount of 

### Low Lying Flooding (No Auto-Detection)


In [41]:
# Write your own custom query
query = "What is the definition of low-lying flooding?"

result = rag_system.generate_response(
    query=query,
    top_k=10,
    layers=None,
    min_confidence="MEDIUM",
    temperature=0.0,
    auto_detect_layers=False
)

rag_system.print_response(result)


🔍 Retrieving top 10 chunks...
✅ Retrieved 10 chunks
📝 Building context prompt...
🤖 Generating response with gpt-4o...

ANSWER
Low-lying flooding is when areas that are at a low elevation, close to sea level, become flooded. This can happen because these areas are naturally more vulnerable to rising sea levels. Imagine a flat piece of land that's just a little bit above the water—when the sea level rises, even a small amount, it can easily spill over into these low-lying areas.

In Hawaii, this is a big concern because many communities and important infrastructure are located near the coast. For example, places like Waikīkī and parts of Honolulu are at risk. As sea levels rise, which scientists project could be up to 3.2 feet by mid-century, these areas could see more frequent and severe flooding. This isn't just about water coming in from the ocean; it can also involve groundwater rising to the surface, making the flooding even worse.

This kind of flooding can affect roads, homes, and

## Test 8: User-Focused Questions

Testing practical user questions with both auto-detection enabled and disabled to compare results.


### Question 1: Why do I need to worry about flooding? (With Auto-Detection)


In [33]:
query = "Why do I need to worry about flooding?"

result = rag_system.generate_response(
    query=query,
    top_k=10,
    layers=None,
    min_confidence="MEDIUM",
    temperature=0.0,
    auto_detect_layers=True
)

rag_system.print_response(result)


🔍 Retrieving top 10 chunks...
✅ Retrieved 10 chunks
📝 Building context prompt...
🤖 Generating response with gpt-4o...

ANSWER
Flooding is something you should definitely be concerned about, especially if you live in or near coastal areas like Honolulu or Waikīkī. As sea levels rise, the risk of flooding increases, and this can have a big impact on homes, roads, and the environment.

Here's why it's important: Research shows that sea levels around Hawaii are rising at a rate of about 1.41 millimeters per year. This might not sound like much, but over time, it adds up. By 2050, sea levels could rise by about 0.4 meters (or roughly 1.3 feet) under an intermediate scenario. By 2100, this could increase to as much as 1.19 meters (nearly 4 feet). This means that areas like Waikīkī and Māpunapuna could see more frequent and severe flooding.

Flooding isn't just about water coming in from the ocean. In Honolulu, for example, flooding can also happen because of rising groundwater, storm-drain b

### Question 1: Why do I need to worry about flooding? (Without Auto-Detection)


In [28]:
query = "Why do I need to worry about flooding?"

result = rag_system.generate_response(
    query=query,
    top_k=10,
    layers=None,
    min_confidence="MEDIUM",
    temperature=0.0,
    auto_detect_layers=False
)

rag_system.print_response(result)


🔍 Retrieving top 10 chunks...
✅ Retrieved 10 chunks
📝 Building context prompt...
🤖 Generating response with gpt-4o...

ANSWER
Flooding is something you should definitely be concerned about, especially if you live in or near coastal areas in Hawaii. As sea levels rise, the risk of flooding increases, and this can have a big impact on your home, community, and daily life.

Here's why it's important: In places like Honolulu and Waikiki, sea level rise is expected to cause significant flooding in the coming decades. For example, research shows that with an intermediate sea level rise scenario, we could see about 1.19 meters (or nearly 4 feet) of sea level rise by 2100. This means that areas that are currently dry could be underwater, especially during high tides or storms.

Flooding isn't just about water coming in from the ocean. In Honolulu, there's something called "groundwater inundation," where rising sea levels push groundwater up to the surface, causing flooding from below. This is 

### Question 2: How do I look at flooding where I live? (With Auto-Detection)


In [29]:
query = "How do I look at flooding where I live?"

result = rag_system.generate_response(
    query=query,
    top_k=10,
    layers=None,
    min_confidence="MEDIUM",
    temperature=0.0,
    auto_detect_layers=True
)

rag_system.print_response(result)


🔍 Retrieving top 10 chunks...
✅ Retrieved 10 chunks
📝 Building context prompt...
🤖 Generating response with gpt-4o...

ANSWER
To see how flooding might affect where you live in Hawaii, you can use the CRC Climate Viewer, which provides detailed maps and projections of different types of flooding due to sea level rise. Here's how you can get started:

1. **Direct Answer**: The CRC Climate Viewer is a tool that lets you explore potential flooding scenarios in your area. You can see how rising sea levels might impact your community by looking at different types of flooding, like passive marine flooding, groundwater inundation, and compound flooding.

2. **Supporting Details**: The tool uses scientific data to simulate flooding scenarios at various sea level rise projections, from 0 to 10 feet. For example, in areas like Waikīkī and Honolulu, research shows that with 3.2 feet of sea level rise, significant flooding could occur, affecting roads and buildings, especially during high tides an

### Question 2: How do I look at flooding where I live? (Without Auto-Detection)


In [30]:
query = "How do I look at flooding where I live?"

result = rag_system.generate_response(
    query=query,
    top_k=10,
    layers=None,
    min_confidence="MEDIUM",
    temperature=0.0,
    auto_detect_layers=False
)

rag_system.print_response(result)


🔍 Retrieving top 10 chunks...
✅ Retrieved 10 chunks
📝 Building context prompt...
🤖 Generating response with gpt-4o...

ANSWER
To see how flooding might affect where you live in Hawaii, you can use the CRC Climate Viewer, which provides detailed maps and projections of different types of flooding. Here's how you can get started:

1. **Direct Answer**: The CRC Climate Viewer is a tool that lets you explore potential flooding scenarios in your area. You can see how sea level rise might impact your community by looking at different types of flooding, like passive marine flooding, groundwater inundation, and compound flooding.

2. **Supporting Details**: The tool uses scientific data to show how much flooding could occur with different levels of sea level rise. For example, you can see projections for 1 foot, 3 feet, or even up to 10 feet of sea level rise. This helps you understand the potential impact on your home, roads, and other important places.

3. **Specific Numbers**: For instance,

### Question 3: Where is flooding on Oahu? (With Auto-Detection)


In [37]:
query = "Where is flooding on Oahu?"

result = rag_system.generate_response(
    query=query,
    top_k=10,
    layers=None,
    min_confidence="MEDIUM",
    temperature=0.0,
    auto_detect_layers=True
)

rag_system.print_response(result)


🔍 Retrieving top 10 chunks...
✅ Retrieved 10 chunks
📝 Building context prompt...
🤖 Generating response with gpt-4o...

ANSWER
Flooding on Oahu is a significant concern, especially as sea levels continue to rise. In particular, areas like Honolulu and Waikiki are vulnerable to different types of flooding. Here's a breakdown of what that means:

1. **Types of Flooding**: Oahu experiences several types of flooding:
   - **Groundwater Inundation**: This happens when the groundwater rises to the surface, which is a big issue in low-lying areas like Honolulu. As sea levels rise, groundwater levels also rise, leading to flooding even without rain.
   - **Compound Flooding**: This occurs when multiple factors, like heavy rain and high tides, combine to cause more severe flooding. For example, during the December 2021 Kona storm, heavy rainfall, king tides, and storm surges led to significant flooding in areas like Kona Moku.
   - **Drainage Backflow**: In places like Waikiki, storm drains can 

### Question 3: Where is flooding on Oahu? (Without Detection)

In [38]:
query = "Where is flooding on Oahu?"

result = rag_system.generate_response(
    query=query,
    top_k=10,
    layers=None,
    min_confidence="MEDIUM",
    temperature=0.0,
    auto_detect_layers=False
)

rag_system.print_response(result)

🔍 Retrieving top 10 chunks...
✅ Retrieved 10 chunks
📝 Building context prompt...
🤖 Generating response with gpt-4o...

ANSWER
Flooding on Oahu is a significant concern, especially as sea levels continue to rise. Here's a breakdown of where and how flooding is affecting the island:

1. **Honolulu and Waikīkī**: These areas are particularly vulnerable to multiple types of flooding. Research indicates that with sea level rise, places like Waikīkī could experience regular flooding, especially during high tides and storms. This is due to a combination of direct marine flooding, storm-drain backflow, and groundwater inundation. For instance, with 3.2 feet of sea level rise, significant parts of Waikīkī could be underwater, affecting roads and buildings.

2. **Groundwater Inundation**: In Honolulu, groundwater flooding is a major issue. This happens when the groundwater rises to the surface, which can occur even without rain or storms. It's expected to be one of the most problematic types of 